# Building a Catalog

In [4]:
from classes.source_catalog import GetCnilCatalog

url = 'https://www.data.gouv.fr/api/1/organizations/534fff61a3a7292c64a77d59/catalog'
headers = {'accept': 'application/json'}
url_add = 'https://www.data.gouv.fr/fr/organizations/cnil/datasets.csv'
instance1 = GetCnilCatalog(url, headers, url_add)
data = instance1.fetch_data_from_api()
data = data['@graph']
table_name = 'title'
download_url = 'downloadURL'
table_id = 'identifier' 
file_format= 'format'
last_update= 'modified' 
accessURL = '@id'
df_catalog = instance1.response_to_dataframe(data=data, table_name=table_name, download_url=download_url, table_id=table_id, file_format=file_format, last_update=last_update, accessURL=accessURL)
df_dataset = instance1.load_additional_info()
df_catalog = instance1.identify_datasets_info()
df_catalog = instance1.merge_additional_info()
instance1.save_to_csv('source_cnil_catalog')

Request is a success: 200
CSV file has been loaded to this path data/catalog/source_cnil_catalog_2024-04-19.csv


# Uploading Files to GCS

In [5]:
# from a dataframe (must have a dest_blob name)

from classes.gcs_processor import GCSProcessor
import os
from datetime import date

today = date.today()
bucket_name = 'cnil_csv'
credential_path = 'cred/service_account_local_py.json'
init2 = GCSProcessor(bucket_name = bucket_name, credentials_path= credential_path)
init2.create_bucket()
file_paths = [df_catalog]
dest_folder = 'raw'
dest_blobs = [f'source_cnil_catalog_{today}.csv']
init2.upload_local_to_gcs(file_paths=file_paths, dest_folder=dest_folder, dest_blobs=dest_blobs, date=today)

Bucket already exists.
2024-04-19/raw/source_cnil_catalog_2024-04-19.csv is uploaded to 2024-04-19/raw/source_cnil_catalog_2024-04-19.csv.


In [ ]:
# from a local file (doesn't require a dest_blob name, it will be the same as the file name)

from classes.gcs_processor import GCSProcessor
import os
from datetime import date

today = date.today()

bucket_name = 'cnil_csv'
credential_path = 'cred/service_account_local_py.json'
init2 = GCSProcessor(bucket_name = bucket_name, credentials_path= credential_path)
init2.create_bucket()
file_paths = [f'data/catalog/source_cnil_catalog_{today}.csv']
dest_folder = 'raw'
dest_blob = [f'source_cnil_catalog_{today}.csv']
init2.upload_local_to_gcs(file_paths=file_paths, dest_folder=dest_folder, date=today)

In [ ]:
# from a URL (doesn't require a dest_blob name, it will be the same as the file name)

from classes.gcs_processor import GCSProcessor
import os
from datetime import date

today = date.today()

bucket_name = 'cnil_csv'
credentials_path = 'cred/service_account_local_py.json'
init2 = GCSProcessor(bucket_name = bucket_name, credentials_path= credentials_path)
init2.create_bucket()
urls = ['https://www.data.gouv.fr/fr/organizations/cnil/datasets.csv', 'https://www.data.gouv.fr/fr/datasets/r/0f678674-4327-4c4d-8819-b6f508b41d0e']
dest_folder = 'raw'
dest_blobs = ['datasets.csv', 'plaintes.csv']
init2.dl_and_up_from_URLs(urls=urls, dest_folder=dest_folder, dest_blobs=dest_blobs, date=today)

# Downloading Files from Catalog

In [1]:
# from a local file to a local file

from classes.download_catalog_content import DlCatalogContentLocal
from datetime import date
from classes.download_catalog_content import GdprSanctionsScrapper

today = date.today()
catalog_path = f'data/catalog/source_cnil_catalog_{today}.csv'

instance3 = DlCatalogContentLocal(catalog_path=catalog_path)
instance3.get_tables()
instance3.zip_files()

zip_file = f'data/raw_datasets_{today}.zip'

scrapper = GdprSanctionsScrapper()
zip_file = scrapper.add_to_zip(zip_file)

Error when downloading table Organismes ayant désigné un(e) délégué(e) à la protection des données (DPD/DPO) : Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?


KeyboardInterrupt: 

In [1]:
# In a IO object

from classes.gcs_processor import GCSProcessor
from datetime import date
import pandas as pd
import os
from classes.download_catalog_content import GdprSanctionsScrapper

today = date.today()

gcs_bucket_name = 'cnil_csv'
credentials_path = 'cred/service_account_local_py.json'
blob_catalog = f"{today}/raw/source_cnil_catalog_{today}.csv"

instance1 = GCSProcessor(bucket_name=gcs_bucket_name, credentials_path=credentials_path)
files = instance1.download_files_from_catalog(catalog_path=blob_catalog)
zip_file = instance1.create_zip_from_files(files)

scrapper = GdprSanctionsScrapper()
zip_file = scrapper.add_to_zip(zip_file)

file_paths = [zip_file]
dest_folder = 'raw'
dest_blobs = ['raw_datasets.zip']
instance1.upload_local_to_gcs(file_paths=file_paths, dest_folder=dest_folder, dest_blobs=dest_blobs, date=today)

Current file downloading: les-deliberations-de-la-cnil/CNIL: les délibérations de la Commission nationale de l'informatique et des libertés_2024-04-19.xml
Current file downloading: traitements-de-donnees-personnelles-declares-a-la-cnil-avant-le-25-mai-2018/Les traitements de données personnelles déclarés à la CNIL entre 1979 et le 24 mai 2018_2024-04-11.csv
File Organismes ayant désigné un(e) délégué(e) à la protection des données (DPD/DPO) does not have a download URL.
Current file downloading: organismes-ayant-designe-un-e-delegue-e-a-la-protection-des-donnees-dpd-dpo/opencnil-organismes-avec-dpo.xlsx_2024-04-08.xlsx
Current file downloading: organismes-ayant-designe-un-e-delegue-e-a-la-protection-des-donnees-dpd-dpo/opencnil-organismes-avec-dpo.csv_2024-04-08.csv
Current file downloading: traitements-de-donnees-personnelles-declares-a-la-cnil-depuis-le-25-mai-2018/Formalités préalables reçues par la CNIL depuis le 25 mai 2018_2024-04-08.csv
File Budget de la CNIL does not have a dow

/Users/benjamindupaquier/Documents/projets_persos/Pipeline/classes/download_catalog_content.py:206: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 206 of the file /Users/benjamindupaquier/Documents/projets_persos/Pipeline/classes/download_catalog_content.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(url_content)


 file 2024-04-19/raw/raw_datasets.zip uploaded to GCS successfully to 2024-04-19/raw/raw_datasets.zip.


# Prep data to upload to BQ

In [12]:
from classes.gcs_processor import GCSProcessor
from datetime import date

today = date.today()

gcs_bucket_name = 'cnil_csv'
credentials_path = 'cred/service_account_local_py.json'

instance1 = GCSProcessor(bucket_name=gcs_bucket_name, credentials_path=credentials_path)
blob_name_zip = f'{today}/raw/raw_datasets.zip'
zip_file = instance1.get_zip_file_object(blob_name_zip)

In [13]:
import pandas as pd
from classes.prep_data import PrepDataCnilBQ

instance5 = PrepDataCnilBQ(zip_file)
dfs = instance5.process_dfs(zip_file)
zip_io = instance5.save_dfs_to_zip(dfs)
errors_log = instance5.errors_backlog

current: les-deliberations-de-la-cnil/CNIL: les délibérations de la Commission nationale de l'informatique et des libertés_2024-04-19.xml
---------------------------------------------------
les-deliberations-de-la-cnil/CNIL: les délibérations de la Commission nationale de l'informatique et des libertés_2024-04-19.xml
<zipfile.ZipExtFile name="les-deliberations-de-la-cnil/CNIL: les délibérations de la Commission nationale de l'informatique et des libertés_2024-04-19.xml" mode='r' compress_type=deflate>
les-deliberations-de-la-cnil/CNIL: les délibérations de la Commission nationale de l'informatique et des libertés_2024-04-19.xml
opened df, return from open_df
les-deliberations-de-la-cnil/CNIL: les délibérations de la Commission nationale de l'informatique et des libertés_2024-04-19.xml not processed!
---------------------------------------------------
current: traitements-de-donnees-personnelles-declares-a-la-cnil-avant-le-25-mai-2018/Les traitements de données personnelles déclarés à l

/Users/benjamindupaquier/anaconda3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(98227, 26)
opened df, return from open_df
More rows than columns, no need to transpose
Re-execution completed.
The column_formatter method worked perfectly.
organismes-ayant-designe-un-e-delegue-e-a-la-protection-des-donnees-dpd-dpo/opencnil-organismes-avec-dpo.xlsx_2024-04-08.xlsx processed successfully!
---------------------------------------------------
current: organismes-ayant-designe-un-e-delegue-e-a-la-protection-des-donnees-dpd-dpo/opencnil-organismes-avec-dpo.csv_2024-04-08.csv
---------------------------------------------------
organismes-ayant-designe-un-e-delegue-e-a-la-protection-des-donnees-dpd-dpo/opencnil-organismes-avec-dpo.csv_2024-04-08.csv
<zipfile.ZipExtFile name='organismes-ayant-designe-un-e-delegue-e-a-la-protection-des-donnees-dpd-dpo/opencnil-organismes-avec-dpo.csv_2024-04-08.csv' mode='r' compress_type=deflate>
organismes-ayant-designe-un-e-delegue-e-a-la-protection-des-donnees-dpd-dpo/opencnil-organismes-avec-dpo.csv_2024-04-08.csv
.csv found
file: <zipfil

/Users/benjamindupaquier/anaconda3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(18907, 12)
opened df, return from open_df
More rows than columns, no need to transpose
Re-execution completed.
The column_formatter method worked perfectly.
traitements-de-donnees-personnelles-declares-a-la-cnil-depuis-le-25-mai-2018/Formalités préalables reçues par la CNIL depuis le 25 mai 2018_2024-03-25.xlsx processed successfully!
---------------------------------------------------
current: sanctions-prononcees-par-la-cnil/opencnil-sanctions-depuis-2019-maj-nov-2023.csv_2023-11-24.csv
---------------------------------------------------
sanctions-prononcees-par-la-cnil/opencnil-sanctions-depuis-2019-maj-nov-2023.csv_2023-11-24.csv
<zipfile.ZipExtFile name='sanctions-prononcees-par-la-cnil/opencnil-sanctions-depuis-2019-maj-nov-2023.csv_2023-11-24.csv' mode='r' compress_type=deflate>
sanctions-prononcees-par-la-cnil/opencnil-sanctions-depuis-2019-maj-nov-2023.csv_2023-11-24.csv
.csv found
file: <zipfile.ZipExtFile name='sanctions-prononcees-par-la-cnil/opencnil-sanctions-depuis-2019

/Users/benjamindupaquier/anaconda3/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


(430, 7)
opened df, return from open_df
More rows than columns, no need to transpose
Re-execution completed.
The column_formatter method worked perfectly.
controles-realises-par-la-cnil/Liste des contrôles réalisés par la CNIL en 2016_2017-03-30.xlsx processed successfully!
---------------------------------------------------
current: controles-realises-par-la-cnil/Liste des contrôles réalisés par la CNIL en 2015_2016-05-03.csv
---------------------------------------------------
controles-realises-par-la-cnil/Liste des contrôles réalisés par la CNIL en 2015_2016-05-03.csv
<zipfile.ZipExtFile name='controles-realises-par-la-cnil/Liste des contrôles réalisés par la CNIL en 2015_2016-05-03.csv' mode='r' compress_type=deflate>
controles-realises-par-la-cnil/Liste des contrôles réalisés par la CNIL en 2015_2016-05-03.csv
.csv found
file: <zipfile.ZipExtFile name='controles-realises-par-la-cnil/Liste des contrôles réalisés par la CNIL en 2015_2016-05-03.csv' mode='r' compress_type=deflate>
Ex

/Users/benjamindupaquier/anaconda3/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


(496, 6)
opened df, return from open_df
More rows than columns, no need to transpose
Re-execution completed.
The column_formatter method worked perfectly.
controles-realises-par-la-cnil/Liste des contrôles réalisés par la CNIL en 2015_2016-05-03.xlsx processed successfully!
---------------------------------------------------
current: controles-realises-par-la-cnil/Liste des contrôles réalisés par la CNIL en 2014_2015-06-15.csv
---------------------------------------------------
controles-realises-par-la-cnil/Liste des contrôles réalisés par la CNIL en 2014_2015-06-15.csv
<zipfile.ZipExtFile name='controles-realises-par-la-cnil/Liste des contrôles réalisés par la CNIL en 2014_2015-06-15.csv' mode='r' compress_type=deflate>
controles-realises-par-la-cnil/Liste des contrôles réalisés par la CNIL en 2014_2015-06-15.csv
.csv found
file: <zipfile.ZipExtFile name='controles-realises-par-la-cnil/Liste des contrôles réalisés par la CNIL en 2014_2015-06-15.csv' mode='r' compress_type=deflate>
Ex

In [16]:
dfs[40]['df']

,annee,type_de_controle,nom_de_lorganisme_controle_ou_nom_de_domaine,lieu_de_controle,departement,secteur_dactivite
0,2014,Loi 1978,Groupon France,Paris,75,Commerce
1,2014,Loi 1978,Groupon France,Courbevoie,92,Commerce
2,2014,Loi 1978,Lycée Marcelin Berthelot,Saint-Maur des Fossés,94,Education/culture/sport
3,2014,Loi 1978,Jive Squad,Gentilly,94,Commerce
4,2014,Loi 1978,Conseil général de l'Hérault,Montpellier,34,Santé/ social
...,...,...,...,...,...,...
416,2014,Contrôles en ligne,iut.univ-avignon.fr (Université d'Avignon et d...,Paris,75,Education/culture/sport
417,2014,Contrôles en ligne,clermont-ferrand.fr,Paris,75,Collectivités locales
418,2014,Contrôles en ligne,u-bordeaux.fr,Paris,75,Commerce
419,2014,Contrôles en ligne,www.ville-chateaudun.fr (Commune De Chateaudun),Paris,75,Collectivités locales


In [9]:
import pandas as pd

df = pd.read_csv('gdpr_fr_sanctions_2024-04-19.csv', sep=';')
df

,date,organisme_type,manquements,decision,theme
0,23/01/2023,SOCIETE DE CONSEIL EN SYSTEMES ET LOGICIELS IN...,\nDéfaut de coopération avec la CNIL\n\t\t\tC...,\nAmende de 5 000 euros et injonction\n,NaN
1,08/02/2023,COMMUNE (procédure simplifiée),Obligation de désigner un délégué à la protect...,Amende de 5 000 euros et injonction,NaN
2,08/02/2023,MEDECIN GENERALISTE (procédure simplifiée),\nNon respect du droit d'accès\n\t\t\tDéfaut d...,\nAmende de 3 000 euros et injonction\n,NaN
3,08/02/2023,SOCIETE EXERCANT UNE ACTIVITE DE DETAIL D'HABI...,Défaut de coopération avec la CNIL,Amende de 10 000 euros et injonction,NaN
4,03/03/2023,SOCIETE EXERCANT UNE ACTIVITE DE SECURITE PRIV...,\nNon respect du principe de minimisation des ...,Amende de 15 000 euros,NaN
...,...,...,...,...,...
210,06/10/2011,SOCIETE DE GESTION IMMOBILIERE,"Commentaires excessifs, données de santé et op...",Avertissement non public,Immobilier
211,06/10/2011,TRANSACTION ET GESTION DE BIENS IMMOBILIERS,"Formalités préalables, défaut de réponse",Avertissement non public,Immobilier
212,21/09/2011,ANNUAIRE,"Collecte déloyale, défaut de mise à jour, non ...",Avertissement public,aspiration de données sur des réseaux sociaux
213,21/07/2011,PARTI POLITIQUE,"Sécurité, confidentialité",Avertissement + procédure d'urgence,Elections


In [3]:
errors_log = instance5.errors_backlog
errors_log

,path,error
0,organismes-ayant-designe-un-e-delegue-e-a-la-p...,unnamed columns
0,organismes-ayant-designe-un-e-delegue-e-a-la-p...,Drop empty columns
1,traitements-de-donnees-personnelles-declares-a...,ParserError and Error tokenizing data. C error...
1,traitements-de-donnees-personnelles-declares-a...,Drop empty columns
1,traitements-de-donnees-personnelles-declares-a...,Column name autre_duree_:_precisez__15_ans_apr...
...,...,...
39,controles-realises-par-la-cnil/Liste des contr...,Drop empty columns
40,controles-realises-par-la-cnil/Liste des contr...,Drop empty columns
41,sanctions/gdpr_eu_sanctions_2024-04-19.csv,ParserError and Error tokenizing data. C error...
41,sanctions/gdpr_eu_sanctions_2024-04-19.csv,Drop empty columns


In [5]:
from classes.gcs_processor import GCSProcessor

bucket_name = 'cnil_csv'
cred_path = 'cred/service_account_local_py.json'
init2 = GCSProcessor(bucket_name=bucket_name, credentials_path=credentials_path)
init2.create_bucket()
file_paths = [zip_io]
dest_folder = 'prep'
dest_blobs = ['prep_gbq_datasets.zip']
instance1.upload_local_to_gcs(file_paths=file_paths, dest_folder=dest_folder, dest_blobs=dest_blobs, date=today)

Bucket already exists.
 file 2024-04-19/prep/prep_gbq_datasets.zip uploaded to GCS successfully to 2024-04-19/prep/prep_gbq_datasets.zip.


In [ ]:
# extract and upload a zip from a blob list. Adapt/filter the blob list to fit your needs

from classes.gcs_processor import GCSProcessor

bucket_name = 'cnil_csv'
cred_path = 'cred/service_account_local_py.json'
init2 = GCSProcessor(bucket_name=bucket_name, credentials_path=credentials_path)
prefix = 'prep'
blobs = init2.list_blobs(prefix=prefix)
init2.extract_and_upload_selection(blobs = blobs, folder_name='prep/extracted', date=today)

# GCS to GCP 

In [1]:
from classes.gcs_processor import GCSProcessor
from datetime import date

today = date.today()

gcs_bucket_name = 'cnil_csv'
credentials_path = 'cred/service_account_local_py.json'
folder = "2024-04-18/prep"

instance1 = GCSProcessor(bucket_name=gcs_bucket_name, credentials_path=credentials_path)
list_blob = instance1.list_blobs(folder)
zip_file = instance1.get_zip_file_object(list_blob[0].name)

cnil_csv


In [2]:
from classes.gcs_to_gcp import FromGCStoGBQ

# usage exemple
credentials_path = 'cred/service_account_local_py.json'
project_id = 'cnil-392113'
dataset_name = 'raw_data'

processor_bq = FromGCStoGBQ(credentials_path, project_id, dataset_name)
processor_bq.create_dataset()
processor_bq.upload_zipio_to_bq(zip_file)

Created dataset (or already exists) cnil-392113.raw_data
---------------------
organismes_ayant_designe_un_e_delegue_e_a_la_protection_des_donnees_dpd_dpo/opencnil_organismes_avec_dpo_xlsx_2024_04_08_xlsx.csv
opencnil_organismes_avec_dpo_xlsx_2024_04_08_xlsx.csv
2024-04-08
xlsx
opencnil_organismes_avec_dpo_xlsx
this is the table name:  cnil-392113.raw_data.opencnil_organismes_avec_dpo_xlsx
---------------------


1it [00:06,  6.97s/it]


organismes_ayant_designe_un_e_delegue_e_a_la_protection_des_donnees_dpd_dpo/opencnil_organismes_avec_dpo_xlsx_2024_04_08_xlsx.csv is uploaded to cnil-392113.raw_data.opencnil_organismes_avec_dpo_xlsx
---------------------
traitements_de_donnees_personnelles_declares_a_la_cnil_depuis_le_25_mai_2018/Formalités_préalables_reçues_par_la_CNIL_depuis_le_25_mai_2018_2024_04_08_csv.csv
Formalités_préalables_reçues_par_la_CNIL_depuis_le_25_mai_2018_2024_04_08_csv.csv
2024-04-08
csv
formalites_prealables_recues_par_la_cnil_depuis_le_25_mai_2018
this is the table name:  cnil-392113.raw_data.formalites_prealables_recues_par_la_cnil_depuis_le_25_mai_2018
---------------------


1it [00:05,  5.32s/it]


traitements_de_donnees_personnelles_declares_a_la_cnil_depuis_le_25_mai_2018/Formalités_préalables_reçues_par_la_CNIL_depuis_le_25_mai_2018_2024_04_08_csv.csv is uploaded to cnil-392113.raw_data.formalites_prealables_recues_par_la_cnil_depuis_le_25_mai_2018
---------------------
budget_de_la_cnil_1/opencnil_budget_depuis_2000_maj_mars_2024_csv_2024_03_29_csv.csv
opencnil_budget_depuis_2000_maj_mars_2024_csv_2024_03_29_csv.csv
2024-03-29
csv
opencnil_budget_depuis_2000_maj_mars_2024_csv
this is the table name:  cnil-392113.raw_data.opencnil_budget_depuis_2000_maj_mars_2024_csv
---------------------


...

controles_realises_par_la_cnil/Liste_des_contrôles_réalisés_par_la_CNIL_en_2015_2016_05_03_csv.csv is uploaded to cnil-392113.raw_data.liste_des_controles_realises_par_la_cnil_en_2015
---------------------
controles_realises_par_la_cnil/Liste_des_contrôles_réalisés_par_la_CNIL_en_2015_2016_05_03_xlsx.csv
Liste_des_contrôles_réalisés_par_la_CNIL_en_2015_2016_05_03_xlsx.csv
2016-05-03
xlsx
liste_des_controles_realises_par_la_cnil_en_2015
this is the table name:  cnil-392113.raw_data.liste_des_controles_realises_par_la_cnil_en_2015
---------------------


1it [00:03,  3.42s/it]


controles_realises_par_la_cnil/Liste_des_contrôles_réalisés_par_la_CNIL_en_2015_2016_05_03_xlsx.csv is uploaded to cnil-392113.raw_data.liste_des_controles_realises_par_la_cnil_en_2015
---------------------
controles_realises_par_la_cnil/Liste_des_contrôles_réalisés_par_la_CNIL_en_2014_2015_06_15_xlsx.csv
Liste_des_contrôles_réalisés_par_la_CNIL_en_2014_2015_06_15_xlsx.csv
2015-06-15
xlsx
liste_des_controles_realises_par_la_cnil_en_2014
this is the table name:  cnil-392113.raw_data.liste_des_controles_realises_par_la_cnil_en_2014
---------------------


1it [00:02,  2.59s/it]

controles_realises_par_la_cnil/Liste_des_contrôles_réalisés_par_la_CNIL_en_2014_2015_06_15_xlsx.csv is uploaded to cnil-392113.raw_data.liste_des_controles_realises_par_la_cnil_en_2014


# Building catalog from prep_data

In [1]:
from classes.prep_data import ZipFileProcessor

gcs_bucket_name = 'cnil_csv'
credential_path = 'cred/service_account_local_py.json'
zip_blob_name = '2024-02-17/prep/prep_datasets.zip'
output_folder_name = '2024-02-17/'+ 'prep'
instance4 = ZipFileProcessor(gcs_bucket_name, credential_path, zip_blob_name, output_folder_name)
zip_file = instance4.get_zip_file_object()

ImportError: cannot import name 'ZipFileProcessor' from 'classes.prep_data' (/Users/benjamindupaquier/Documents/projets_persos/Pipeline/classes/prep_data.py)

In [ ]:
from classes.source_catalog import CustomCatalog
import io

instance8 = CustomCatalog('cred/service_account_local_py.json')
df = instance8.create_catalog_gcs(zip_file)
df

In [ ]:
from classes.file_to_gcs import FromFileToGCS

bucket_name = 'cnil_csv'
cred_path = 'cred/service_account_local_py.json'
init2 = FromFileToGCS(bucket_name, cred_path)
init2.create_bucket()
file_paths = [csv_output]
dest_folder = 'prep'
dest_blob = ['prepdata_cnil_catalog_2024-02-17.csv']
init2.local_to_gcs(file_paths, dest_folder, dest_blob)

# Building catalog from BQ raw_data

In [1]:
from classes.source_catalog import CustomCatalog

credential_path = 'cred/service_account_local_py.json'
dataset_name = 'raw_data'
project_id = 'cnil-392113'
instance8 = CustomCatalog(credentials_path=credential_path, project_id=project_id, dataset_name=dataset_name)
df = instance8.bq_raw_catalog()

Getting BigQuery modified dates...
Done.


In [3]:
from classes.gcs_to_gcp import FromGCStoGBQ

credentials_path = 'cred/service_account_local_py.json'
project_id = 'cnil-392113'
dataset_name = 'catalog_data'
table_name = 'cnil_catalog_bq'

processor_bq = FromGCStoGBQ(credentials_path, project_id, dataset_name)
processor_bq.create_dataset()
processor_bq.df_to_bq(df, table_name)

Created dataset (or already exists) cnil-392113.catalog_data


1it [00:03,  3.81s/it]

DataFrame is uploaded to cnil-392113.catalog_data.cnil_catalog_bq
